In [10]:
import pandas as pd 
import numpy as np
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [11]:
people_data = pd.read_csv("./data_model/people_wiki.csv")
people_data.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


In [33]:
brad = people_data[people_data['name'] == 'Brad Pitt']
brad['text']


24426    william bradley brad pitt born december 18 196...
Name: text, dtype: object

In [16]:
def sortedbow(wordstring):
    wordlist = wordstring.split()
    bow = {}
    for w in wordlist:
        bow[w] = 0
    for w in wordlist:
        bow[w] += 1 
    print (sorted (bow.items(), key=lambda x: x[1], reverse=True))

In [18]:
sortedbow(str(brad['text']))

[('24426', 1), ('william', 1), ('bradley', 1), ('brad', 1), ('pitt', 1), ('born', 1), ('december', 1), ('18', 1), ('196...', 1), ('Name:', 1), ('text,', 1), ('dtype:', 1), ('object', 1)]


In [35]:


from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
word_count = count_vectorizer.fit_transform(people_data['text'])
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(word_count)



In [36]:
brad_words_count = count_vectorizer.transform(['Brad Pitt'])
brad_tfidf = tfidf_transformer.transform(brad_words_count)
res = neigh.kneighbors(brad_tfidf, return_distance=False)
print (res)

[[37466 32112 37965 24426 29417 16488 13967  9480  3282  5510]]


In [38]:
for i in res: 
    print (people_data.iloc[i])

                                                     URI  \
37466          <http://dbpedia.org/resource/Warren_Pitt>   
32112  <http://dbpedia.org/resource/Charles_Redding_P...   
37965  <http://dbpedia.org/resource/Bill_Pitt_(racing...   
24426            <http://dbpedia.org/resource/Brad_Pitt>   
29417        <http://dbpedia.org/resource/Joe_Luxbacher>   
16488         <http://dbpedia.org/resource/Tim_Grgurich>   
13967  <http://dbpedia.org/resource/Brad_Williams_(co...   
9480           <http://dbpedia.org/resource/Brad_Nichol>   
3282          <http://dbpedia.org/resource/Naomi_Wilson>   
5510            <http://dbpedia.org/resource/Freddie_Fu>   

                            name  \
37466                Warren Pitt   
32112       Charles Redding Pitt   
37965  Bill Pitt (racing driver)   
24426                  Brad Pitt   
29417              Joe Luxbacher   
16488               Tim Grgurich   
13967   Brad Williams (comedian)   
9480                 Brad Nichol   
3282           